In [1]:
from tensorflow.keras.datasets import cifar10

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt

# Load CIFAR-10 dataset
(train_images, _), (_, _) = tf.keras.datasets.cifar10.load_data()
train_images = (train_images - 127.5) / 127.5  # Normalize to the range [-1, 1]

# Generator model
def build_generator(latent_dim):
    model = models.Sequential()
    model.add(layers.Dense(4 * 4 * 256, use_bias=False, input_dim=latent_dim))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Reshape((4, 4, 256)))

    model.add(layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.2))

    model.add(layers.Conv2DTranspose(64, (4, 4), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.2))

    model.add(layers.Conv2DTranspose(3, (4, 4), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))

    return model

# Discriminator model
def build_discriminator(img_shape):
    model = models.Sequential()
    model.add(layers.Conv2D(64, (4, 4), strides=(2, 2), padding='same', input_shape=img_shape))
    model.add(layers.LeakyReLU(alpha=0.2))

    model.add(layers.Conv2D(128, (4, 4), strides=(2, 2), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.2))

    model.add(layers.Conv2D(256, (4, 4), strides=(2, 2), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.2))

    model.add(layers.Flatten())
    model.add(layers.Dense(1, activation='sigmoid'))

    return model

# Create and compile the discriminator
img_shape = (32, 32, 3)
discriminator = build_discriminator(img_shape)
discriminator.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5), metrics=['accuracy'])

# Create the generator
latent_dim = 10
generator = build_generator(latent_dim)

# Create the GAN
discriminator.trainable = False
gan_input = tf.keras.Input(shape=(latent_dim,))
fake_image = generator(gan_input)
gan_output = discriminator(fake_image)
gan = models.Model(gan_input, gan_output)
gan.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5))

# Training loop
def train_gan(epochs, batch_size):
    batch_count = train_images.shape[0] // batch_size

    for e in range(epochs):
        for _ in range(batch_count):
            noise = np.random.normal(0, 1, size=(batch_size, latent_dim))
            generated_images = generator.predict(noise)

            real_images = train_images[np.random.randint(0, train_images.shape[0], batch_size)]

            labels_real = np.ones((batch_size, 1))
            labels_fake = np.zeros((batch_size, 1))

            d_loss_real = discriminator.train_on_batch(real_images, labels_real)
            d_loss_fake = discriminator.train_on_batch(generated_images, labels_fake)

            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            noise = np.random.normal(0, 1, size=(batch_size, latent_dim))
            labels_gan = np.ones((batch_size, 1))
            g_loss = gan.train_on_batch(noise, labels_gan)

        print(f"Epoch {e+1}/{epochs}, [D loss: {d_loss[0]} | D accuracy: {10 * d_loss[1]}] [G loss: {g_loss}]")

# Train the GAN
train_gan(epochs=10, batch_size=64)

# Generate fake images
def generate_fake_images(samples=10):
    noise = np.random.normal(0, 1, size=(samples, latent_dim))
    generated_images = generator.predict(noise)
    
    generated_images = 0.5 * generated_images + 0.5  # Rescale to [0, 1]

    for i in range(samples):
        plt.subplot(2, 5, i+1)
        plt.imshow(generated_images[i])
        plt.axis('off')
    plt.show()

# Generate and visualize fake images
generate_fake_images()


2/2 [==============================] - 0s 72ms/step


2/2 [==============================] - 0s 66ms/step


2/2 [==============================] - 0s 70ms/step


2/2 [==============================] - 0s 65ms/step


2/2 [==============================] - 0s 75ms/step
Epoch 1/10, [D loss: 6.0672797189909033e-05 | D accuracy: 10.0] [G loss: 0.00016346196935046464]
2/2 [==============================] - 0s 88ms/step


...

2/2 [==============================] - 0s 175ms/step


2/2 [==============================] - 1s 243ms/step


2/2 [==============================] - 1s 236ms/step


2/2 [==============================] - 1s 231ms/step
Epoch 8/10, [D loss: 4.049408031114865e-09 | D accuracy: 10.0] [G loss: 2.3111667246666912e-07]
2/2 [==============================] - 0s 198ms/step


2/2 [==============================] - 1s 271ms/step


In [ ]:

from IPython.display import Image
%matplotlib inline

In [ ]:
from numpy import asarray
from keras.models import Sequential
from keras.layers import UpSampling2D
#define input
x=asarray([[1,2],[3,4]])
print('Input Array:\n',x)
print()
#reshape
x=x.reshape((1,2,2,1))
print('After Reshape:\n',x,x.shape)
#define model
model=Sequential()
model.add(UpSampling2D(input_shape=(2,2,1)))
#summarize the model
model.summary()
ypred=model.predict(x)
ypred=ypred.reshape((4,4))
print('Upsampled Array:\n',ypred)

In [ ]:
from numpy import asarray
from keras.models import Sequential
from keras.layers import UpSampling2D
from keras.layers import Dense,Conv2D,Reshape
model=Sequential()
model.add(Dense(128*5*5,input_dim=100))
model.add(Reshape((5,5,128)))
model.add(UpSampling2D())
model.add(Conv2D(1,(3,3),padding='same'))
model.summary()
from numpy import asarray
from keras.models import Sequential
from keras.layers import UpSampling2D
from keras.layers import Conv2DTranspose
x=asarray([[1,2],[3,4]])
print('Input Array:\n',x)
print()
#reshape
x=x.reshape((1,2,2,1))
print('After Reshape:\n',x,x.shape)
#define model
model=Sequential()
model.add(Conv2DTranspose(1,(1,1),strides=(2,2),input_shape=(2,2,1)))
#summarize the model
model.summary()
weights=[asarray([[[[1]]]]),asarray([0])]
model.set_weights(weights)
ypred=model.predict(x)
ypred=ypred.reshape((4,4))
print('Transposed Array:\n',ypred)

In [ ]:
from keras.datasets.mnist import load_data
(xtrain,ytrain),(xtest,ytest)=load_data()
print('Train Shape:\t',xtrain.shape,ytrain.shape)
print()
print('Test Shape:\t',xtest.shape,ytest.shape)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(xtrain[1],cmap='gray')


In [ ]:

for i in range(5):
 plt.subplot(5,5,1+i)
 plt.axis('off')
 plt.imshow(xtrain[i],cmap='gray_r')
plt.show()

In [ ]:

#Import all required libraries
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Conv2D,Flatten,Dropout
from keras.layers import LeakyReLU
from keras.optimizers import Adam
from keras.utils import plot_model

In [ ]:

#Import all required libraries
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Conv2D,Flatten,Dropout
from keras.layers import LeakyReLU
from keras.optimizers import Adam
from keras.utils import plot_model

def discriminator(in_shape=(28,28,1)):
    model=Sequential()
    model.add(Conv2D(64,(3,3),strides=(2,2),padding='same',input_shape=in_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Conv2D(64,(3,3),strides=(2,2),padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Flatten())
    model.add(Dense(1,activation='sigmoid'))
    opt=Adam(learning_rate=0.0002,beta_1=0.5)
    model.compile(loss='binary_crossentropy',optimizer=opt,metrics=['accuracy'])
    return model
model=discriminator()
model.summary()

In [ ]:

def train_discriminator(model,dataset,n_iter=100,n_batch=256):
    half_batch=int(n_batch//22)
    for i in range(n_iter):
        xreal,yreal=generate_real_samples(dataset,half_batch)
    _,real_acc=model.train_on_batch(xreal,yreal)
    xfake,yfake=generate_fake_samples(half_batch)
    _,fake_acc=model.train_on_batch(xfake,yfake)
    print('>%d real=%.0f%% fake=%.0f%%'%(i+1,real_acc*100,fake_acc*100))
    xfake,yfake=generate_fake_samples(half_batch)
    _,fake_acc=model.train_on_batch(xfake,yfake)
    print('>%d real=%.0f%% fake=%.0f%%'%(i+1,real_acc*100,fake_acc*100))

In [ ]:

from numpy import expand_dims,ones,zeros
def load_real_samples():
    (xtrain,_),(_,_)=load_data()
    x=expand_dims(xtrain,axis=-1)
    x=x.astype('float32')
    x=x/255.
    return x


In [ ]:

model=discriminator()
#load image data
dataset=load_real_samples()

In [ ]:
from numpy.random import rand,randint
def generate_real_samples(dataset,n_samples):
    ix=randint(0,dataset.shape[0],n_samples)
    x=dataset[ix]
    y=ones((n_samples,1))
    return x,y

In [ ]:
generate_real_samples(load_real_samples(),n_samples=10)


In [ ]:
def generate_fake_samples(n_samples):
    x=rand(28*28*n_samples)
    x=x.reshape(n_samples,28,28,1)
    y=zeros((n_samples,1))
    return x,y


In [ ]:
generate_fake_samples(10)


In [ ]:
train_discriminator(model,dataset)